In [1]:
from crossref.restful import Journals
import pandas as pd
def data_api(issn,journal_name):
    authors_list = []
    title_list = []
    affiliations_list = []
    published_dates_list=[]
    journal_names_list=[]
    subjects_list=[]
    volumes_list=[]
    issues_list=[]
    URL_list=[]
    DOI_list=[]
    count = 0
    journals = Journals()
    results = journals.works(issn=issn)
    search = results.filter(from_pub_date="2000").select(["container-title","author","DOI","URL",'subject','volume',"title",'issue',"published-print"])
    total = search.count()
    for item in search:
        authors = [f"{author.get('given', '')} {author.get('family', '')}" for author in item.get('author', [])]
        if len(authors) >0:
            authors_list.append('; '.join(authors))
            # Get the affiliations
            affiliations = [aff.get('name', '') for author in item.get('author', []) for aff in author.get('affiliation', [])]
            affiliations_list.append('; '.join(affiliations))
            title_list.append(item.get('title', '')[0])
            published_date = '-'.join(str(part) for part in item.get('published-print', {}).get('date-parts', [[]])[0])
            published_dates_list.append(published_date)
            journal_names_list.append(item.get('container-title', [''])[0])
            subjects_list.append('; '.join(item.get('subject', [])))
            volumes_list.append(item.get('volume', ''))
            issues_list.append(item.get('issue', ''))
            URL_list.append(item.get('URL',''))
            DOI_list.append(item.get('DOI',''))
            count +=1
            print("load success",count,"/",total)
    data = {
            'Authors': authors_list,
            'Title' : title_list,
            'Affiliations': affiliations_list,
            'Published Date': published_dates_list,
            'Journal Name ': journal_names_list, 
            'Subjects': subjects_list,
            'Volume': volumes_list,
            'Issue': issues_list,
            'DOI': DOI_list,
            'URL': URL_list
    
        }
    df = pd.DataFrame(data)
        
    # Save the DataFrame to a CSV file
    df.to_csv(f'{journal_name}_api.csv', index=True)

In [52]:
#formal
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
def data_check(journal_name,redo = False,start = 0):
    headers_list = [
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Upgrade-Insecure-Requests': '1'
    },
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Upgrade-Insecure-Requests': '1'
    },
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Upgrade-Insecure-Requests': '1'
    },
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Upgrade-Insecure-Requests': '1'
    },
    {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Upgrade-Insecure-Requests': '1'
    }
    ]
    # 创建cloudscraper对象
    scraper = cloudscraper.create_scraper()
    if redo == False:
        df = pd.read_csv(f"{journal_name}_api.csv", index_col=0)
    else:
        df = pd.read_csv(f"{journal_name}.csv", index_col=0)
    count = start
    total = len(df["DOI"])
    
    for index, row in df.iloc[start:].iterrows():
        url = "https://onlinelibrary.wiley.com/doi/abs/"+row["DOI"]
        print(url)
        affiliation = row["Affiliations"]
    
        # 检查Affiliations列是否为空
        if pd.isna(affiliation) or df.iloc[index]["Title"] == df.iloc[index-1]["Title"]:
            print("open")
            try:
                headers = random.choice(headers_list)
                # 导航到URL
                response = scraper.get(url)
                random_wait_object = random.uniform(1,100)
                if count%100 == random_wait_object:
                    random_wait_time = random.uniform(60,80)
                else:
                    random_wait_time = random.uniform(5,12)
                time.sleep(random_wait_time)
                if response.status_code == 200:
                    print("open success")
                    # 获取网页源代码
                    page_source = response.text
                
                    # 使用Beautiful Soup解析HTML
                    soup = BeautifulSoup(page_source, 'html.parser')
                
                    # 查找和提取作者信息
                    # 获取 citation_author
                    authors = [meta['content'] for meta in soup.find_all('meta', {'name': 'citation_author'})]
                
                    # 获取 citation_author_institution
                    author_institutions = [meta['content'] for meta in soup.find_all('meta', {'name': 'citation_author_institution'})]
                
                    # 获取 citation_publication_date
                    publication_date = soup.find('meta', {'name': 'citation_publication_date'})['content'] if soup.find('meta', {'name': 'citation_publication_date'}) else None
                
                    # 获取 citation_title
                    title = soup.find('meta', {'name': 'citation_title'})['content'] if soup.find('meta', {'name': 'citation_title'}) else None
                    authors_str = '; '.join(authors)
                    author_institutions_str = '; '.join(author_institutions)
                    # 更新行
                    df.at[index, 'Authors'] = authors_str
                    df.at[index, 'Affiliations'] = author_institutions_str
                    df.at[index, 'Published Date'] = publication_date
                    df.at[index, 'Title'] = title
                    print(df.iloc[index])
                    print('success',count,"/",total)
                else:
                    print(f"Failed to retrieve: {response.status_code}",count,"/",total)
            except Exception as e:
                print(f"An error occurred: {e}",count,"/",total)
            finally:
                count += 1
        else:
            count += 1
            print(f"Skipping {url} as Affiliations is not empty.",count,"/",total)
            
    # 保存更新后的 CSV
    df.to_csv(f'{journal_name}.csv',index=True)


In [10]:
data_api('0741-6261','Rand Journal of Economics')

load success 1 / 981
load success 2 / 981
load success 3 / 981
load success 4 / 981
load success 5 / 981
load success 6 / 981
load success 7 / 981
load success 8 / 981
load success 9 / 981
load success 10 / 981
load success 11 / 981
load success 12 / 981
load success 13 / 981
load success 14 / 981
load success 15 / 981
load success 16 / 981
load success 17 / 981
load success 18 / 981
load success 19 / 981
load success 20 / 981
load success 21 / 981
load success 22 / 981
load success 23 / 981
load success 24 / 981
load success 25 / 981
load success 26 / 981
load success 27 / 981
load success 28 / 981
load success 29 / 981
load success 30 / 981
load success 31 / 981
load success 32 / 981
load success 33 / 981
load success 34 / 981
load success 35 / 981
load success 36 / 981
load success 37 / 981
load success 38 / 981
load success 39 / 981
load success 40 / 981
load success 41 / 981
load success 42 / 981
load success 43 / 981
load success 44 / 981
load success 45 / 981
load success 46 / 9

In [48]:
data_check('Quantitative Economics',redo=False,start=0)

https://onlinelibrary.wiley.com/doi/abs/10.3982/qe1781
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe1781 as Affiliations is not empty. 2 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe459
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe459 as Affiliations is not empty. 3 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe517
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe517 as Affiliations is not empty. 4 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe436
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe436 as Affiliations is not empty. 5 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe465
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe465 as Affiliations is not empty. 6 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe1751
Skipping https://onlinelibrary.wiley.com/doi/abs/10.3982/qe1751 as Affiliations is not empty. 7 / 394
https://onlinelibrary.wiley.com/doi/abs/10.3982/qe1692
Skipping ht

In [51]:
redo_list = ["Econometrica","Theoretical Economics"]
for name in redo_list:
    data_check(name,redo = True,start=0)

https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2005.00602.x
Skipping https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2005.00602.x as Affiliations is not empty. 1 / 548
https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2004.00533.x
open
open success
Authors                                        Xueli Tang; Ian King
Title             A Comment on Roland Benabou's “Tax and Educati...
Affiliations      1Dept. of Economics, University of Auckland, 3...
Published Date                                           2005/05/01
Journal Name                                           Econometrica
Subjects                                 Economics and Econometrics
Volume                                                           73
Issue                                                             3
DOI                                10.1111/j.1468-0262.2005.00602.x
URL               http://dx.doi.org/10.1111/j.1468-0262.2005.006...
Name: 0, dtype: object
success 1 / 5